# **Interactive Visualization of Indian Covid Data**

*A Notebook which does Exploratory Data Analysis  of India Covid Data*

**1. Importing Required Libraries for EDA**

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import json
from IPython.display import clear_output
import plotly.graph_objects as go

1. Interactive Bubble Chart For Date Wise Visualization of Covid Cases.

In [ ]:
df = pd.read_csv('../input/covid19-corona-virus-india-dataset/complete.csv')
a = list(df['Death'])
for i in range(len(a)):
    try:
        a[i] = int(a[i])
    except:
        a[i] = 0
a = pd.DataFrame(a)
df['Death' ] =a
df['Name of State / UT'].replace('Telengana','Telangana',inplace = True)
df['Name of State / UT'].replace('Telangana***','Telangana',inplace = True)
fig = px.scatter(df.sort_values('Name of State / UT'),x = 'Date' , y = 'Death' ,  size = 'New cases' , color = 'Name of State / UT',
                 size_max = 30,)

fig.update_xaxes(
    rangeslider_visible=True,showgrid  =False,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.update_yaxes(showgrid = False)
fig.update_layout(
title_text='COVID Data State Wise Description')

fig.show()

**2. Indian Map Visualization to Visualize Intensity of COVID in Indian States**

In [ ]:
india_states = json.load(open('../input/india-map-geojson/states_india.geojson'))
df = pd.read_csv('../input/india-map-geojson/a.csv')
state_id_map = {}
for feature in india_states['features']:
    feature['id'] = feature['properties']['state_code']
    state_id_map[feature['properties']['st_nm']] = feature['id']
df['State'].replace('Andaman and Nicobar Islands','Andaman & Nicobar Island',inplace = True)
df['State'].replace('Jammu and Kashmir','Jammu & Kashmir',inplace = True)
df['State'].replace('Dadra and Nagar Haveli and Daman and Diu','Dadara & Nagar Havelli',inplace = True)
df['State'].replace('Delhi','NCT of Delhi',inplace = True)
df['State'].replace('Arunachal Pradesh','Arunanchal Pradesh',inplace = True)
df.drop([17],inplace = True)
df['id'] = df['State'].apply(lambda x : state_id_map[x])
fig = go.Figure(go.Choroplethmapbox(geojson=india_states, locations=df.id, z=df.Confirmed,
                                    colorscale="twilight",
                                    marker_opacity=0.5,
                                   ))
fig.update_layout(title_text='Confirmed Cases State wise',
    mapbox_style="carto-positron",
                  mapbox_zoom=3, mapbox_center = {"lat": 20.5937, "lon": 78.9629})
fig.show()


**3. A Tree Map to Show Positive to Death Ratio For Each City Provided the Sate From Menu (Run The Code)**

In [ ]:
district_level=pd.read_csv('../input/covid19-corona-virus-india-dataset/district_level_latest.csv')
district_level.drop(range(728,800),inplace=True)
district_level.drop(0,inplace=True)
district_level=district_level[['State','District','Confirmed','Deceased']]
district_level=district_level[['State','District','Confirmed','Deceased']]
deaths=district_level['Deceased'].to_numpy()
confirm=district_level['Confirmed'].to_numpy()
d_by_p=np.zeros(deaths.shape)

for i in range(deaths.shape[0]):
    d_by_p[i]=deaths[i]/confirm[i]*100
ser=pd.Series(d_by_p)
district_level['death_by_positive']=ser
district_level.dropna(inplace=True)
for i,state in enumerate(district_level['State'].unique()):
    print(f'{i}  - {state}')
s=district_level['State'].unique()[int(input())]
clear_output(wait=True)
fig = px.treemap(district_level[district_level['State']==s], path=['State','District'], values='death_by_positive')
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25),title_text='Death,Positive Ratio For Each City in State')
fig.show()

**4. A Parallel Categories Map which visualizes the categories of Patients Affected By COVID in India**

In [ ]:
df= pd.read_csv('../input/covid19-corona-virus-india-dataset/patients_data.csv')
df.dropna(subset=['Gender','Nationality','Detected State', 'Current Status'],inplace=True)
df['Current Status'].unique()
dic={
    'Recovered':0,
    'Hospitalized':0.5,
    'Deceased':1
}
d = pd.DataFrame()
d_arr=df['Current Status'].to_numpy()

new_list=[]
for i in d_arr:
    new_list.append(dic[i])
d=pd.Series(new_list)
df.reset_index(inplace=True)
df['Colour_Number']=d
ds = go.parcats.Dimension(
values=df['Detected State'],
categoryorder='category ascending', label="Detected State"
)



gn = go.parcats.Dimension(values=df['Gender'], label="Gender")



cs = go.parcats.Dimension(
values=df['Current Status'], label="Current Status",
)
nt = go.parcats.Dimension(
values=df['Nationality'], label="Nationality",
)
# Create parcats trace
color = df['Colour_Number'];
colorscale = [[0, 'mediumseagreen'],[0.5,'lightsteelblue'], [1, 'blue']];



fig = go.Figure(data = [go.Parcats(dimensions=[ds,gn,cs,nt],
line={'color': color, 'colorscale': colorscale},
labelfont={'size': 15, 'family': 'Times'},
tickfont={'size': 10, 'family': 'Times'},
arrangement='freeform')])


fig.update_layout(
title_text='Patient Information Visualization')
fig.show()

**5. Interactive Visualization of Important COVID Data**

In [ ]:
df=pd.read_csv('../input/covid19-corona-virus-india-dataset/complete.csv')
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['New cases'], name="New cases",
                    line_shape='linear'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Total Confirmed cases'], name="Total Confirmed cases",
                    line_shape='linear'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Cured/Discharged/Migrated'], name="Cured/Discharged/Migrated",
                    line_shape='linear'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['New recovered'], name="New Recovered",
                    line_shape='linear'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Death'], name="Death",
                    line_shape='linear'))
fig.update_xaxes(
    rangeslider_visible=True,showgrid = False,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.update_yaxes(showgrid = False)
fig.show()

**6. A Heat Map Which Visualizes the Intensity of New Cases in States Date Wise**

In [ ]:

df=pd.read_csv('../input/covid19-corona-virus-india-dataset/complete.csv')
fig = go.Figure(data=go.Heatmap(
                   y=df['Name of State / UT'],
                   z=df['New cases'],
                   x=df['Date'],colorscale='viridis',
                   hoverongaps = False))
fig.update_xaxes(
    rangeslider_visible=True,showgrid = False
)
fig.update_yaxes(showgrid = False)
fig.show()